In [1]:
from datasets import load_from_disk

dataset = load_from_disk(r"C:\Studies\Datasets\XLAM_60k")

c:\Studies\Softwares\Anaconda\envs\sacenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset['train']

Dataset({
    features: ['id', 'query', 'answers', 'tools'],
    num_rows: 60000
})

In [4]:
from model import get_model

model, tokenizer = get_model()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W1228 17:02:02.588000 18728 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Studies\Softwares\Anaconda\envs\sacenv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:348: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE_TORCH}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA GeForce RTX 3070 Laptop GPU. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.9.1+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]
[accelerate.big_modeling|WARNING]Some parameters are on the meta device because they were offloaded to the cpu.
Unsloth 2025.11.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
from unsloth.chat_templates import get_chat_template

In [8]:
# Initialize the tokenizer with the chat template and mapping
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3", 
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True,        # Maps <|im_end|> to <|eot_id|> instead
)

In [17]:
dataset['train'][0]

{'id': 0,
 'query': 'Where can I find live giveaways for beta access and games?',
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

In [16]:
dataset['train']['query']

Column(['Where can I find live giveaways for beta access and games?', "I need to understand the details of the Ethereum blockchain for my cryptocurrency project. Can you fetch the details for 'ethereum'?", "What is the T3MA for 'ETH/BTC' using a 1h interval and a time period of 14?", "List titles originally aired on networks '1' and '8', released after 2010, sorted by release date in descending order.", 'Fetch the competitor standings for the recently concluded stage 98765.', ...])

In [25]:
def formatting_prompts_func(examples):
    convos = []
    
    # Iterate through each item in the batch (examples are structured as lists of values)
    for query, tools, answers in zip(examples['query'], examples['tools'], examples['answers']):
        tool_user = {
            "content": f"You are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n{tools}",
            "role": "system"
        }
        ques_user = {
            "content": f"{query}",
            "role": "user"
        }
        assistant = {
            "content": f"{answers}",
            "role": "assistant"
        }
        convos.append([tool_user, ques_user, assistant])

    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

formatting_prompts_func(dataset['train'][:5])

{'text': ['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere can I find live giveaways for beta access and games?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]<|eot_id|>',
  '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful a

In [20]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 60000/60000 [00:02<00:00, 26662.73 examples/s]


In [23]:
dataset['train']['text']

Column(['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant with access to the following tools or function calls. Your task is to produce a sequence of tools or function calls necessary to generate response to the user utterance. Use the following tools or function calls as required:\n[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhere can I find live giveaways for beta access and games?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]<|eot_id|>', '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assis